# Public Dataset Preprocessing

# Step 1
- Convert all the flow txt files (output of Tranalyzer) into CSV files

In [2]:
import csv
import os

directory = r'E:\UNSW\DNS_results'  # change the directory accordingly (output of Tranalyzer)

for filename in os.listdir(directory):
    if filename.endswith('_flows.txt'):
        file_path = os.path.join(directory, filename)

        with open(file_path, 'r') as file:
            lines = file.read().splitlines()  # read the data from the text file

        headers = lines[0].split("\t")  # headers
        content = [line.split("\t") for line in lines[1:]]  # content

        csv_filename = filename.replace('_flows.txt', '') + '.csv'  # corresponding csv file name
        csv_file_path = os.path.join(directory, csv_filename)

        # Convert the txt files to csv files
        with open(csv_file_path, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile, delimiter=',')
            
            # Write the data
            writer.writerow(headers)
            for row in content:
                writer.writerow(row)

        print(f"Data from {filename} written to {csv_filename}")

print("All files processed.")

Data from 16-09-23_flows.txt written to 16-09-23.csv
Data from 16-09-24_flows.txt written to 16-09-24.csv
Data from 16-09-25_flows.txt written to 16-09-25.csv
Data from 16-09-26_flows.txt written to 16-09-26.csv
Data from 16-09-27_flows.txt written to 16-09-27.csv
Data from 16-09-28_flows.txt written to 16-09-28.csv
Data from 16-09-29_flows.txt written to 16-09-29.csv
Data from 16-09-30_flows.txt written to 16-09-30.csv
Data from 16-10-01_flows.txt written to 16-10-01.csv
Data from 16-10-02_flows.txt written to 16-10-02.csv
Data from 16-10-03_flows.txt written to 16-10-03.csv
Data from 16-10-04_flows.txt written to 16-10-04.csv
Data from 16-10-05_flows.txt written to 16-10-05.csv
Data from 16-10-06_flows.txt written to 16-10-06.csv
Data from 16-10-07_flows.txt written to 16-10-07.csv
Data from 16-10-08_flows.txt written to 16-10-08.csv
Data from 16-10-09_flows.txt written to 16-10-09.csv
Data from 16-10-10_flows.txt written to 16-10-10.csv
Data from 16-10-11_flows.txt written to 16-10-

# Step 2: Data Cleaning - Only keep the following types of traffic
 - IoT devices communicate with endpoints outside the local network
 - DHCP and DNS when IoT devices communicate with the gateway

In [1]:
import pandas as pd
import os

# Change the directory accordingly
source_directory = r'E:\UNSW\DNS_results'
destination_directory = r'E:\UNSW\New_Filtered_data'

os.makedirs(destination_directory, exist_ok=True)

# Non-IoT devices MAC addresses in UNSW
Non_IoT_mac_addresses = {'40:f3:08:ff:1e:da', '74:2f:68:81:69:42', 'ac:bc:32:d4:6f:2f', 
                         'b4:ce:f6:a7:a3:c2', 'd0:a6:37:df:a1:e1', 'f4:5c:89:93:cc:85', 
                         '08:21:ef:3b:fc:e3', 'e8:ab:fa:19:de:4f', '30:8c:fb:b6:ea:45'}  # the last two are useless IoT devices in UNSW

# IoT devices MAC addresses in UNSW
IoT_mac_addresses = {'d0:52:a8:00:67:5e', '44:65:0d:56:cc:d3', '70:ee:50:18:34:43', 'f4:f2:6d:93:51:f1', 
                      '00:16:6c:ab:6b:88', '30:8c:fb:2f:e4:b2', '00:62:6e:51:27:2e', 'e8:ab:fa:19:de:4f', 
                      '00:24:e4:11:18:a8', 'ec:1a:59:79:f4:89', '50:c7:bf:00:56:39', '74:c6:3b:29:d7:1d', 
                      'ec:1a:59:83:28:11', '18:b4:30:25:be:e4', '70:ee:50:03:b8:ac', '00:24:e4:1b:6f:96', 
                      '74:6a:89:00:2e:25', '00:24:e4:20:28:c6', 'd0:73:d5:01:83:08', '18:b7:9e:02:20:44', 
                      'e0:76:d0:33:bb:85', '70:5a:0f:e4:9b:c0', '30:8c:fb:b6:ea:45'}

# Data cleaning
for filename in os.listdir(source_directory):
    if filename.endswith('.csv'):
        file_path = os.path.join(source_directory, filename)

        df = pd.read_csv(file_path, low_memory=False)  # read the file into a dataframe

        # Filter out Non-IoT device traffic
        df_filtered = df[~df['srcMac'].isin(Non_IoT_mac_addresses) & ~df['dstMac'].isin(Non_IoT_mac_addresses)]

        # Filter out traffic between IoT devices
        df_filtered = df_filtered[~(df_filtered['srcMac'].isin(IoT_mac_addresses) & df_filtered['dstMac'].isin(IoT_mac_addresses))]

        # Filter out flows with multiple MAC addresses
        df_filtered = df_filtered[~df_filtered['srcMac'].str.contains(';') & ~df_filtered['dstMac'].str.contains(';')]
        
        # Only keep flows where either 'srcMac' or 'dstMac' is an IoT device
        df_filtered = df_filtered[df_filtered['srcMac'].isin(IoT_mac_addresses) | df_filtered['dstMac'].isin(IoT_mac_addresses)]
        
        # ONLY KEEP TRAFFIC WITH OUTSIDE ENDPOINTS AND DHCP, DNS FLOWS
        # Add a new column showing the endpoints' countries
        df_filtered['IpCountry'] = df_filtered.apply(lambda row: row['dstIPCC'] if row['%dir'] == 'A' else row['srcIPCC'], axis=1)
        
        # Keep flows where IP country is not a number (meaning this flow is with the endpoint outside the local network) and DNS/DHCP flows
        df_filtered = df_filtered[(df_filtered['IpCountry'].str.isnumeric() == False) & (df_filtered['IpCountry'] != "-") |
                                  (df_filtered['dstPortClassN'] == 53) | (df_filtered['dstPortClassN'] == 67)]

        filtered_csv_file_path = os.path.join(destination_directory, filename)
        df_filtered.to_csv(filtered_csv_file_path, index=False)  # write the dataframe back to the CSV file

        print(f"Filtered data from {filename} written to {filtered_csv_file_path}")

print("All files processed.")

Filtered data from 16-09-23.csv written to E:\UNSW\New_Filtered_data\16-09-23.csv
Filtered data from 16-09-24.csv written to E:\UNSW\New_Filtered_data\16-09-24.csv
Filtered data from 16-09-25.csv written to E:\UNSW\New_Filtered_data\16-09-25.csv
Filtered data from 16-09-26.csv written to E:\UNSW\New_Filtered_data\16-09-26.csv
Filtered data from 16-09-27.csv written to E:\UNSW\New_Filtered_data\16-09-27.csv
Filtered data from 16-09-28.csv written to E:\UNSW\New_Filtered_data\16-09-28.csv
Filtered data from 16-09-29.csv written to E:\UNSW\New_Filtered_data\16-09-29.csv
Filtered data from 16-09-30.csv written to E:\UNSW\New_Filtered_data\16-09-30.csv
Filtered data from 16-10-01.csv written to E:\UNSW\New_Filtered_data\16-10-01.csv
Filtered data from 16-10-02.csv written to E:\UNSW\New_Filtered_data\16-10-02.csv
Filtered data from 16-10-03.csv written to E:\UNSW\New_Filtered_data\16-10-03.csv
Filtered data from 16-10-04.csv written to E:\UNSW\New_Filtered_data\16-10-04.csv
Filtered data fr

# Step 3: Add four new features for each IoT device
- Number of visited endpoints
- Number of visited NTP endpoints
- Number of visited endpoints' countries
- Number of visited NTP endpoints' countries

In [14]:
import pandas as pd
import os


# Calculate the number of visited (NTP) endpoints - by DOMAIN NAMES rather than IP addresses
def calculate_num_of_endpoints(df, IoT_mac_addresses):
    unique_info_per_srcMac = {}  # store the new features for each unqiue IoT device (per srcMac)

    # Iterate over each unique srcMac (each IoT device)
    for srcMac in df['srcMac'].unique():
        if srcMac not in IoT_mac_addresses:
            continue
        
        # Initialize sets to store features for this IoT device
        unique_tuples_set = set()
        unique_dnsQnames = set()
        unique_dnsQnames_with_ntp = set()

        # Find unique visited dstIPs for each IoT device
        unique_dstIPs = df[df['srcMac'] == srcMac]['dstIP'].unique()

        # Iterate over each unique dstIP
        for dstIP in unique_dstIPs:
            # Exclude dstIPs starting with 192.168.1 and 0.0.0.0
            if dstIP.startswith("192.168.1") or dstIP == "0.0.0.0":
                continue

            # Extract the corresponding dstPortClass for this dstIP
            dstPortClass_rows = df[df['dstIP'] == dstIP]['dstPortClass']
            dstPortClass = dstPortClass_rows.iloc[0] if not dstPortClass_rows.empty else None

            # Check if this dstIP is in any dns4Aaddress or dns6Aaddress from DNS flows
            matched_dns4 = df[df['dns4Aaddress'].str.contains(dstIP, na=False)]
            matched_dns6 = df[df['dns6Aaddress'].str.contains(dstIP, na=False)]
            matched_dns = pd.concat([matched_dns4, matched_dns6]).drop_duplicates()

            if not matched_dns.empty:
                # If found, process each match
                for _, row in matched_dns.iterrows():
                    dnsQname = row['dnsQname']
                    dstMac = row['dstMac']
                    
                    # Process tuples only when this DNS flow belongs to the corresponding IoT device
                    if srcMac == dstMac:
                        tuple_to_print = (srcMac, dstIP, dnsQname, dstMac, dstPortClass)
                        if tuple_to_print not in unique_tuples_set:
                            unique_tuples_set.add(tuple_to_print)
                            unique_dnsQnames.add(dnsQname)
                            
                            # Count NTP endpoints
                            if dstPortClass == "ntp":
                                unique_dnsQnames_with_ntp.add(dnsQname)

        # Store the number of visited domain names and the number of visited NTP domain names for each IoT devcie
        unique_info_per_srcMac[srcMac] = {
            'total_unique_dnsQnames': len(unique_dnsQnames),
            'unique_dnsQnames_with_ntp': len(unique_dnsQnames_with_ntp)
        }
    
    return unique_info_per_srcMac


# Calculate the number of visited endpoints' countries
def calculate_num_of_endpoints_countries(df, IoT_mac_addresses):
    # Create a new column for grouping, using dstMac if srcMac is not the IoT device
    df['group_mac'] = df.apply(lambda row: row['dstMac'] if row['srcMac'] not in IoT_mac_addresses else row['srcMac'], axis=1)

    filtered_df = df[(df['dstIPCC'] != '-') & (~df['dstIPCC'].str.isnumeric())]
    num_endpoints_countries = filtered_df.groupby('group_mac')['dstIPCC'].nunique().reset_index(name='Num_of_endpoints_countries')

    result = df.merge(num_endpoints_countries, on='group_mac', how='left')
    result['Num_of_endpoints_countries'] = result['Num_of_endpoints_countries'].fillna(0).astype(int)
    
    return result.drop(columns=['group_mac'])


# Calculate the number of visited NTP endpoints' countries
def calculate_num_of_ntp_endpoints_countries(df, IoT_mac_addresses):
    # Create a new column for grouping, using dstMac if srcMac is not the IoT device
    df['group_mac'] = df.apply(lambda row: row['dstMac'] if row['srcMac'] not in IoT_mac_addresses else row['srcMac'], axis=1)

    filtered_df = df[(df['dstPortClass'] == 'ntp') & (df['dstIPCC'] != '-') & (~df['dstIPCC'].str.isnumeric())]  # check for NTP endpoints
    num_ntp_endpoints_countries = filtered_df.groupby('group_mac')['dstIPCC'].nunique().reset_index(name='Num_of_ntp_endpoints_countries')

    result = df.merge(num_ntp_endpoints_countries, on='group_mac', how='left')
    result['Num_of_ntp_endpoints_countries'] = result['Num_of_ntp_endpoints_countries'].fillna(0).astype(int)
    
    return result.drop(columns=['group_mac'])

# Step 4: Data Labeling

In [7]:
# Label each flow, mapping each one to an IoT device
def label_data(df, mapping, IoT_mac_addresses):
    
    def get_label(mac):
        return mapping.get(mac, None)
    
    # Label the flow either by srcMac or dstMac
    df['label'] = df.apply(lambda row: get_label(row['srcMac']) if row['srcMac'] in IoT_mac_addresses else get_label(row['dstMac']), axis=1)
    
    return df

# Main Function

In [8]:
def main():
    # Change the directories accordingly
    source_directory = r'E:\UNSW\New_Filtered_data'
    destination_directory = r'E:\UNSW\Processed_data'
    
    os.makedirs(destination_directory, exist_ok=True)
    
    # IoT MAC addresses
    IoT_mac_addresses = {'d0:52:a8:00:67:5e', '44:65:0d:56:cc:d3', '70:ee:50:18:34:43', 'f4:f2:6d:93:51:f1', 
                         '00:16:6c:ab:6b:88', '30:8c:fb:2f:e4:b2', '00:62:6e:51:27:2e', '00:24:e4:11:18:a8', 
                         'ec:1a:59:79:f4:89', '50:c7:bf:00:56:39', '74:c6:3b:29:d7:1d', 'ec:1a:59:83:28:11', 
                         '18:b4:30:25:be:e4', '70:ee:50:03:b8:ac', '00:24:e4:1b:6f:96', '74:6a:89:00:2e:25', 
                         '00:24:e4:20:28:c6', 'd0:73:d5:01:83:08', '18:b7:9e:02:20:44', 'e0:76:d0:33:bb:85', 
                         '70:5a:0f:e4:9b:c0'}
    
    # Map each IoT device to a label
    label_mapping = {
        'd0:52:a8:00:67:5e': 0,
        '44:65:0d:56:cc:d3': 1,
        '70:ee:50:18:34:43': 2,
        'f4:f2:6d:93:51:f1': 3,
        '00:16:6c:ab:6b:88': 4,
        '30:8c:fb:2f:e4:b2': 5,
        '00:62:6e:51:27:2e': 6,
        '00:24:e4:11:18:a8': 7,
        'ec:1a:59:79:f4:89': 8,
        '50:c7:bf:00:56:39': 9,
        '74:c6:3b:29:d7:1d': 10,
        'ec:1a:59:83:28:11': 11,
        '18:b4:30:25:be:e4': 12,
        '70:ee:50:03:b8:ac': 13,
        '00:24:e4:1b:6f:96': 14,
        '74:6a:89:00:2e:25': 15,
        '00:24:e4:20:28:c6': 16,
        'd0:73:d5:01:83:08': 17,
        '18:b7:9e:02:20:44': 18,
        'e0:76:d0:33:bb:85': 19,
        '70:5a:0f:e4:9b:c0': 20
    }
    
    for filename in os.listdir(source_directory):
        if filename.endswith('.csv'):
            file_path = os.path.join(source_directory, filename)
    
            df = pd.read_csv(file_path, low_memory=False)  # read the csv file as a dataframe
            
            df = label_data(df, label_mapping, IoT_mac_addresses)  # label the data
            
            unique_info = calculate_num_of_endpoints(df, IoT_mac_addresses) # calculate the number of visited (NTP) endpoints

            # Add new columns to the original dataframe, based on the srcMac or dstMac
            df['Number_of_endpoints'] = df.apply(lambda row: unique_info.get(row['srcMac'], {'total_unique_dstIPs': 0})['total_unique_dstIPs'] if row['srcMac'] in IoT_mac_addresses else unique_info.get(row['dstMac'], {'total_unique_dstIPs': 0})['total_unique_dstIPs'], axis=1)
            df['Number_of_ntp_endpoints'] = df.apply(lambda row: unique_info.get(row['srcMac'], {'unique_ntp_dstIPs': 0})['unique_ntp_dstIPs'] if row['srcMac'] in IoT_mac_addresses else unique_info.get(row['dstMac'], {'unique_ntp_dstIPs': 0})['unique_ntp_dstIPs'], axis=1)

            df = calculate_num_of_endpoints_countries(df, IoT_mac_addresses)  # calculate the number of visited endpoints' countries
            df = calculate_num_of_ntp_endpoints_countries(df, IoT_mac_addresses)  # calculate the number of visited NTP endpoints' countries
            
            # Delete useless columns
            columns_to_delete = ['dnsStat', 'dnsHdrOPField', 'dnsHFlg_OpC_RetC', 'dnsCntQu_Asw_Aux_Add', 'dnsAAAqF', 'dnsQname', 'dnsAname', 'dnsAPname', 'dns4Aaddress', 'dns6Aaddress', 'dnsQType', 'dnsQClass', 'dnsAType', 'dnsAClass', 'dnsATTL', 'dnsMXpref', 'dnsSRVprio', 'dnsSRVwgt', 'dnsSRVprt', 'dnsOptStat', 'IpCountry']
            df.drop(columns=columns_to_delete, inplace=True)

            new_csv_path = os.path.join(destination_directory, filename)
            df.to_csv(new_csv_path, index=False)  # updated csv files
        
            print(f"Processed {filename} and saved to {new_csv_path}")

In [9]:
main()

Processed 16-09-23.csv and saved to E:\UNSW\Processed_data_endpoints_IP\16-09-23.csv
Processed 16-09-24.csv and saved to E:\UNSW\Processed_data_endpoints_IP\16-09-24.csv
Processed 16-09-25.csv and saved to E:\UNSW\Processed_data_endpoints_IP\16-09-25.csv
Processed 16-09-26.csv and saved to E:\UNSW\Processed_data_endpoints_IP\16-09-26.csv
Processed 16-09-27.csv and saved to E:\UNSW\Processed_data_endpoints_IP\16-09-27.csv
Processed 16-09-28.csv and saved to E:\UNSW\Processed_data_endpoints_IP\16-09-28.csv
Processed 16-09-29.csv and saved to E:\UNSW\Processed_data_endpoints_IP\16-09-29.csv
Processed 16-09-30.csv and saved to E:\UNSW\Processed_data_endpoints_IP\16-09-30.csv
Processed 16-10-01.csv and saved to E:\UNSW\Processed_data_endpoints_IP\16-10-01.csv
Processed 16-10-02.csv and saved to E:\UNSW\Processed_data_endpoints_IP\16-10-02.csv
Processed 16-10-03.csv and saved to E:\UNSW\Processed_data_endpoints_IP\16-10-03.csv
Processed 16-10-04.csv and saved to E:\UNSW\Processed_data_endpoi